<a href="https://colab.research.google.com/github/koushikkhan/ts_forecasting/blob/master/stock_price_forecasting_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Price Forecasting using LSTM Networks

## Import modules

In [0]:
import os
import math
import re
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta
import pandas_datareader as pdr

In [0]:
from tensorflow.keras.models import Sequential, load_model, model_from_json
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, LSTM, GRU, Bidirectional, Dropout, Activation
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.constraints import NonNeg

## Mount Google Drive (Optional)

In [0]:
# drive.mount('/content/drive', force_remount=True)

## Setup Path (Optional)

In [0]:
# home = os.path.join(os.getcwd(), "drive", "My Drive", "must")
# model_path = os.path.join(home, "model")

## Utility Functions

In [0]:
# prepare LSTM training data
def create_dataset(orig_data_array, target_col_index, n_time_step, n_future_step=1):
    """
    Generate dataset of shape `(n_sample, n_timestep, n_feature)` for LSTM network.

    # Args:
        orig_data_array: numpy.ndarray, source data
        target_col_index: int, index for the target variable within `orig_data_array`
        n_time_step: int, past history size to be considered
        n_future_step: int, future steps to forecast
        
    # Return:
        x: numpy.ndarray, array of shape `(n_sample, n_time_step, n_feature)`
        y: numpy.ndarray, array of shape `(n_sample, n_future_step)`
    """
    assert isinstance(orig_data_array, np.ndarray), "`orig_data_array` is expected to be of numpy ndarray type"
    x, y = [], []
    for i in range(len(orig_data_array) - n_time_step - n_future_step + 1):
        x.append(orig_data_array[i:(i + n_time_step)])
        y.append(
            orig_data_array[
                (i + n_time_step):(i + n_time_step + n_future_step)
            ][:, target_col_index]
        )
    x = np.array(x)
    y = np.array(y).reshape((len(y), n_future_step))
    return x, y


# prepare test data
def create_test_dataset(orig_data_array):
    assert isinstance(orig_data_array, np.ndarray), "`orig_data_array` is expected to be of numpy ndarray type"
    time_steps = orig_data_array.shape[0]
    features = orig_data_array.shape[1]
    x = orig_data_array.reshape((1, time_steps, features))
    return x


# partition data
def split_source_data(data_array, train_prop=0.90):
    # split into train and test sets
    train_size = int(len(data_array) * train_prop)
    if isinstance(data_array, np.ndarray):
        train, valid = data_array[0:train_size,:], data_array[train_size:len(dataset),:]
    if isinstance(data_array, pd.DataFrame):
        train, valid = data_array.iloc[0:train_size, :], data_array.iloc[train_size:, :]
    return train, valid


# convert target/forecasts into original scale
def inverse_scale_target(target_array, scaler_obj, target_index):
    t = (target_array * \
                (scaler_obj.data_max_[target_index] - scaler_obj.data_min_[target_index])) + \
                scaler_obj.data_min_[target_index]
    return t


# train lstm model
def train_lstm_model(train_x, train_y, hidden_size, output_size, dr, lr, batch_size, n_epochs):
    model = Sequential()
    model.add(
            LSTM(
                hidden_size,
                return_sequences=False,
                input_shape=(train_x.shape[1], train_x.shape[2]),
                recurrent_dropout=dr,
                activation='sigmoid'
            )
    )
    model.add(Dropout(dr))
    model.add(Dense(units=output_size, activation='linear'))
    
    model.compile(loss='mean_absolute_error', 
                  metrics=['mse'], optimizer=Adam(lr=lr))
    
    print(model.summary())
        
    history = model.fit(
        train_x, train_y, batch_size=batch_size, epochs=n_epochs, 
        verbose=2
    )
    
    return model, history


# plot model performance history
def plot_model_history(modl_history):
    loss = modl_history.history['loss']
    val_loss = modl_history.history['val_loss']
    epochs = range(len(loss))

    plt.figure()
    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.show()

    mape = modl_history.history['mape']
    val_mape = modl_history.history['val_mape']
    epochs = range(len(mape))

    plt.figure()
    plt.plot(epochs, mape, 'b', label='Training MAPE')
    plt.plot(epochs, val_mape, 'r', label='Validation MAPE')
    plt.title('Training and Validation MAPE')
    plt.legend()
    plt.show()


# compute 'mean-absolute-percentage-error'
def calculate_mape(model, x, y, smoothing_factor=0.5):
    saved_args = locals()
    # preds = (model.predict(x).flatten() * (scaler.data_max_[target_index] - scaler.data_min_[target_index])) + scaler.data_min_[target_index]
    preds = model.predict(x).flatten()
    actuals = y.flatten()
    if 0 in actuals:
        m = np.mean(np.abs((preds - actuals)/(actuals + smoothing_factor))) * 100
    else:
        m = np.mean(np.abs((preds - actuals)/actuals)) * 100
    return m


# concatenate 2d output to 1d output
def convert_target_to_1d(target, scaler_obj, target_col_index):
    target_list = list(target)
    all_except_first_output = target_list[1:]
    first_output_separated = target_list[0]
    remaining_single_outputs = [x[-1] for x in all_except_first_output]
    test_outputs_concatentated = np.hstack((
        first_output_separated, 
        np.array(remaining_single_outputs))
    )
    test_outputs_concatentated = (test_outputs_concatentated * \
                                    (scaler_obj.data_max_[target_col_index] - scaler_obj.data_min_[target_col_index])) + \
                                    scaler_obj.data_min_[target_col_index]
    return test_outputs_concatentated


# create dataframe filled with np.nan
def create_nan_dataframe(nrow, columns):
    emp_df = pd.DataFrame(
        np.nan,
        index=range(0, nrow),
        columns=columns
    )
    return emp_df


def calculate_mape(df):
    actual = df.iloc[:, 1].values
    forecasted = df.iloc[:, 2].values

    if 0 in actual:
        m = np.mean(np.abs((actual-forecasted)/(actual + 0.5))) * 100
    else:
        m = np.mean(np.abs((actual-forecasted)/actual)) * 100
    return m

## Data Exploration and modeling

### Global Parameters for data processing and modeling


In [0]:
# organisation name
org = "GOOG"

# history start and end date
hist_start = datetime(1991, 4, 1)
hist_end = datetime(2020, 4, 16)


# training and validation end date
train_end_date = datetime.strptime("2020-01-31", "%Y-%m-%d")
valid_end_date = datetime.strptime("2020-04-16", "%Y-%m-%d")

# forecasting period start date
# forecast_end_date = datetime.strptime("2020-04-16", "%Y-%m-%d")

# no. of days to look behind
n_history_size = 10

# no. of days to calculate forecast for
n_forecast = 5

# target column index
target_col_index = 3

# lstm hidden dimensionality
hidden_size = 512

# learning rate
lr = 0.001

# dropout rate
dr = 0.3

# batch size
batch_size = 32

# no. of epochs
n_epochs = 25

# model name
model_name = org + "_" + str(hidden_size) + ".json"
model_weighths_name = org + "_" + str(hidden_size) + "_" + "weights.h5"

### Download historical data from Yahoo Finance

In [0]:
dataframe = pdr.get_data_yahoo(symbols=org, start=hist_start, end=hist_end)

# remove 'Date' from index and place it as a column
dataframe.reset_index(inplace=True)

In [0]:
dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%Y-%m-%d")   
dataframe.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2004-08-19,51.835709,47.800831,49.813286,49.982655,44871300.0,49.982655
1,2004-08-20,54.336334,50.062355,50.316402,53.952770,22942800.0,53.952770
2,2004-08-23,56.528118,54.321388,55.168217,54.495735,18342800.0,54.495735
3,2004-08-24,55.591629,51.591621,55.412300,52.239193,15319700.0,52.239193
4,2004-08-25,53.798351,51.746044,52.284027,52.802086,9232100.0,52.802086


### Split data into train and validation parts

In [0]:
train = dataframe.copy()[dataframe['Date'] <= train_end_date]
train_size = train.shape[0]

valid = dataframe.copy()[(dataframe['Date'] > train_end_date) & (dataframe['Date'] <= valid_end_date)]
valid_size = valid.shape[0]

print(f"training data shape: {train.shape}")
print(f"validation data shape: {valid.shape}")

training data shape: (3890, 7)
validation data shape: (52, 7)


### Visualize the actual time series 

In [0]:
# Plot Time Series
plot_df = pd.concat([train, valid], axis=0)
fig = px.line(plot_df , x='Date', y='Close', title=f'Daily Close Price')
fig.show()

### Apply scaling to normalize the dataset

In [0]:
# merge train and validation part before applying scaling
train_valid_df = pd.concat([train, valid], axis=0)
train_valid_df_values = train_valid_df.drop(['Date'], axis=1).values

In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(train_valid_df_values)

In [0]:
# print(dataset.shape)
train_scaled, valid_scaled = dataset[0:train_size, :], dataset[train_size:(train_size+valid_size), :]
print(f'Train part shape: {train_scaled.shape}')
print(f'Validation part shape: {valid_scaled.shape}')

Train part shape: (3890, 6)
Validation part shape: (52, 6)


### Example: LSTM Sample Training Data Creation using `create_dataset()` function

<img src="https://drive.google.com/uc?id=1MKuS_1GymjzRYISmkBKTPH57iNKYl-Gh" height="500" align="right">

### Create LSTM / GRU dataset of shape `(n_sample, n_timestep, n_features)`

In [0]:
trainX, trainY = create_dataset(
    orig_data_array=train_scaled, 
    target_col_index=target_col_index, 
    n_time_step=n_history_size, 
    n_future_step=n_forecast
)

validX, validY = create_dataset(
    orig_data_array=valid_scaled, 
    target_col_index=target_col_index, 
    n_time_step=n_history_size, 
    n_future_step=n_forecast
)

In [0]:
print(f"Shape of training-X data: {trainX.shape}")
print(f"Shape of training-Y data: {trainY.shape}")
print(f"Shape of validation-X data: {validX.shape}")
print(f"Shape of validation-Y data: {validY.shape}")

Shape of training-X data: (3876, 10, 6)
Shape of training-Y data: (3876, 5)
Shape of validation-X data: (38, 10, 6)
Shape of validation-Y data: (38, 5)


### LSTM Architecture

<img src="https://docs.google.com/drawings/d/e/2PACX-1vRCrVnkND45dcIk6-NgJdvZsDvuFnz3JhK4kVxJFjnHdZxE72TrVms7rzuCaqaBDCmq-nk7ESnCBYNN/pub?w=1644&amp;h=1000">

### Execute model training

In [0]:
model_, history_ = train_lstm_model(
    train_x=trainX, 
    train_y=trainY, 
    hidden_size=hidden_size, 
    output_size=n_forecast, 
    dr=dr, 
    lr=lr,
    batch_size=batch_size, 
    n_epochs=25
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1062912   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 1,065,477
Trainable params: 1,065,477
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/25
122/122 - 27s - loss: 0.2048 - mse: 0.0741
Epoch 2/25
122/122 - 27s - loss: 0.1064 - mse: 0.0183
Epoch 3/25
122/122 - 27s - loss: 0.0593 - mse: 0.0060
Epoch 4/25
122/122 - 27s - loss: 0.0421 - mse: 0.0032
Epoch 5/25
122/122 - 27s - loss: 0.0382 - mse: 0.0027
Epoch 6/25
122/122 - 27s - loss: 0.0343 - mse: 0.0022
Epoch 7/25
122/122 - 27s - loss: 0.0321 - mse

### Save Model to Google Drive (Optional)

In [0]:
# serialize model to JSON
# model_json = model_.to_json()
# with open(os.path.join(model_path, model_name), "w") as json_file:
#     json_file.write(model_json)

# # serialize weights to HDF5
# model_.save_weights(os.path.join(model_path, model_weighths_name))
# print("Saved model to disk")

## Prepare output for final visualization

### Load model from Google Drive (Optional)

In [0]:
# # load json and create model
# json_file = open(os.path.join(model_path, model_name), 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model_ = model_from_json(loaded_model_json)

# # load weights into new model
# model_.load_weights(os.path.join(model_path, model_weighths_name))
# print("Loaded model from disk")

Loaded model from disk


### During Training Period

In [0]:
actual_concatentated = convert_target_to_1d(
    target=trainY, 
    scaler_obj=scaler, 
    target_col_index=target_col_index
)

forecast_concatentated = convert_target_to_1d(
    target=model_.predict(trainX), 
    scaler_obj=scaler, 
    target_col_index=target_col_index
) 

In [0]:
output_df_train = create_nan_dataframe(
    nrow=actual_concatentated.shape[0],
    columns=['Actual', 'Fitted']
)

output_df_train['Actual'] = actual_concatentated
output_df_train['Fitted'] = forecast_concatentated

dt_train = train[['Date']].tail(n=actual_concatentated.shape[0]).reset_index()
dt_train.drop(['index'], axis=1, inplace=True)
output_df_train = dt_train.merge(output_df_train.copy(), left_index=True, right_index=True)

In [0]:
# consider last 100 days output
# output_df_train = output_df_train.tail(n=200)
output_df_train = output_df_train
# output_df_train.shape

### Plot Actual and Fitted values during training

In [0]:
output_train = pd.melt(
    output_df_train.copy(), id_vars=['Date'], 
               value_vars=['Actual', 'Fitted'],
               var_name='Price', value_name='Close Price'
)

output_train.shape

(7760, 3)

In [0]:
# Plot actual vs Fitted Time Series for training Period
px.line(output_train, 
        x='Date', 
        y='Close Price', 
        color='Price', 
        title=f'Actual and Fitted Daily Close Prices')

### During Validation Period

In [0]:
actual_concatentated = convert_target_to_1d(
    target=validY, 
    scaler_obj=scaler, 
    target_col_index=target_col_index
)

preds_concatentated = convert_target_to_1d(
    target=model_.predict(validX), 
    scaler_obj=scaler, 
    target_col_index=target_col_index
)

In [0]:
output_df_valid = create_nan_dataframe(
    nrow=actual_concatentated.shape[0],
    columns=['Actual', 'Forecasted']
)

output_df_valid['Actual'] = actual_concatentated
output_df_valid['Forecasted'] = preds_concatentated

dt_valid = valid[['Date']].tail(n=actual_concatentated.shape[0]).reset_index()
dt_valid.drop(['index'], axis=1, inplace=True)
output_df_valid = dt_valid.merge(output_df_valid.copy(), left_index=True, right_index=True)
# output_df_valid.tail()

In [0]:
# extract date using regex
m = re.search(r'\d{4}-\d{2}-\d{2}', str(output_df_valid.iloc[-1, 0]))
valid_last_date = datetime.strptime(m.group(), "%Y-%m-%d") + timedelta(days=1) 
# valid_last_date

### During Forecasting Period

In [0]:
forecast_raw_input = valid.tail(n=n_history_size)
forecast_raw_input_nodate = forecast_raw_input.copy().drop(['Date'], axis=1)
to_be_forecasted = scaler.transform(forecast_raw_input_nodate.values)
to_be_forecasted = create_test_dataset(to_be_forecasted)

In [0]:
# calculate forecast
forecast = convert_target_to_1d(
    target=model_.predict(to_be_forecasted), 
    scaler_obj=scaler, 
    target_col_index=target_col_index
) 

# print(forecast)

In [0]:
output_df_forecast = create_nan_dataframe(
    nrow=forecast.shape[0], 
    columns=['Actual', 'Forecasted']
)

output_df_forecast['Date'] = pd.date_range(valid_last_date, periods=n_forecast, freq='D')
output_df_forecast['Forecasted'] = forecast
# output_df_forecast

In [0]:
# Combine training, validation and forecast dfs'
output_df = pd.concat([output_df_valid, output_df_forecast]).reset_index()
output_df.drop(['index'], axis=1, inplace=True)

### Apply `.melt()` to reshape the final output

In [0]:
output_valid_forecast = pd.melt(
    output_df.copy(), id_vars=['Date'], 
               value_vars=['Actual', 'Forecasted'],
               var_name='Price', value_name='Close Price'
)

output_valid_forecast.shape

(94, 3)

### Plot Actual, Predicted and Forecasted prices

In [0]:
# Plot actual vs Fitted Time Series for Validation Period
px.line(output_valid_forecast, 
        x='Date', 
        y='Close Price', 
        color='Price', 
        title=f'Actual, Fitted and Forecasted Daily Close Prices')

## Calculate Performance Metric: **MAPE**

Formula:

$$
mape = 100 \times \frac{1}{n}\sum_{t=1}^n \left|\frac{actual_t - forecasted_t}{actual_t}\right|
$$

### MAPE in training period

In [0]:
print(f"Training Mape (in percentage): %0.2f " % calculate_mape(output_df_train))

Training Mape (in percentage): 4.54 


### MAPE in Validation Period

In [0]:
print(f"Validation Mape (in percentage): %0.2f " % calculate_mape(output_df_valid))

Validation Mape (in percentage): 6.91 


## Future Scopes



1.   Exploring simple **Gated Recurrent Units** (GRU) sequences
2.   Exploring LSTM / GRU cells inside **encoder-decoder** setup
3.   Using attention mechanism

